# Civil War Book Review scratchpad

https://www.reportlab.com/docs/reportlab-userguide.pdf



In [1]:
from collections import namedtuple
import csv
import re
import os

In [2]:
cwbr_dict = dict()

# name the file you wish to examine [Split_Into_Types/---------.csv]

with open('Split_Into_Types/Hyperlink_in_review.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter="\t", quotechar='"')
    headers = next(csvreader)
    CWBR = namedtuple('CWBR', headers)
    for row in csvreader:
        item = CWBR(*row)
        cwbr_dict[item.ID] = item

In [3]:
longest_item = ('', '')
for k, v in cwbr_dict.items():
    if len(v.Review) > len(longest_item[1]):
        longest_item = (k, v.Review)
        print(k)

5737
6113
5986
6306
4245
6048


In [4]:
def strip_and_replace(text):
    return text.strip().replace(":", "").replace(".", "").replace(",", "").replace('"', '').replace("<p>", "").lower().replace(' ', '')

inside, not_inside = 0, 0
for uid, issue in cwbr_dict.items():
    if issue.Teaser in issue.Title:
        inside += 1
    elif strip_and_replace(issue.Teaser[:-5]) in strip_and_replace(issue.Title):
        inside += 1
    else:
        not_inside += 1
        
#       is Teaser a subset of Title??
#         print("Teaser: {}\nTitle: {}\n\n".format(issue.Teaser, issue.Title))
# print(inside, not_inside)

In [5]:
# how many have Sub_headlines?  How do they Sub_headlines compare to Titles, Headlines?

count = 0
for uri, issue in cwbr_dict.items():
    if len(issue.Sub_headline):
        count += 1
#         print("{}\nS: {}\nH: {}\nT: {}\n\n".format(uri, issue.Sub_headline, issue.Headline, issue.Title))
# print(count)

In [6]:
# How many have a Headline?  Are Headlines a shortened form of the Title?

count = 0
similar = 0
for uri, issue in cwbr_dict.items():
    if len(issue.Headline):
        count += 1
        if strip_and_replace(issue.Headline) in strip_and_replace(issue.Title):
            similar += 1
#             print("{}\nS: {}\nH: {}\nT: {}\n\n".format(uri, issue.Sub_headline, issue.Headline, issue.Title))
# print(count, similar)

In [7]:
# what is our datastructure?  give an example

# item = cwbr_dict['1434']
# print(dir(item))
# for field in item._fields:
#     print("{}\t\t{}".format(field, getattr(item, field)[:100]))

In [8]:
# how many have a Teaser?  print all the Teasers to screen.

count = 0
for k, v in cwbr_dict.items():
    if len(v.Teaser):
        count += 1
#         print(v.Teaser)
# print(count)

In [10]:
# can I identify an image url?

path_regex = re.compile('src="(?P<path>[\S]+)"')
imagepath = path_regex.search(item.Image)
print(item.Image)
webpath = imagepath.group('path')
filepath = os.path.join('/media/francis/S/')


<img hspace=6 align=LEFT width=161 height=245 alt="Product Cover" border=0 src="/civilwarbookreview/images/jackets/ReluctantRebels.jpg">


In [11]:
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

In [1]:
# a styling with justified lines & indentation of some number of points.

styles=getSampleStyleSheet()
styles.add(ParagraphStyle(name='Justify',
                          alignment=TA_JUSTIFY,
                          firstLineIndent=25))


NameError: name 'getSampleStyleSheet' is not defined

In [13]:
# script to turn the html into a pdf.  
# several different variations are included, to challenge our dataset -- to see which have errors.

# This one crashes on the first failure.

def do_thing(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)

    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')

    if len(split_strings) == 1:
        p = Paragraph(split_strings[0], styles["Justify"])
        template.append(p)
    else:
        for i in split_strings:
            p = Paragraph(i, styles["Justify"])
            template.append(p)
            template.append(Spacer(1,0.2*inch))

    doc.build(template)


In [ ]:
# This one logs which ones fail, then continues to the next one.

def do_thing_log_fails(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    try:
        if len(split_strings) == 1:
            p = Paragraph(split_strings[0], styles["Justify"])
            template.append(p)
        else:
            for i in split_strings:
                if len(i):
                    p = Paragraph(i, styles["Justify"])
                    template.append(p)
                    template.append(Spacer(1,0.2*inch))
                else:
                    pass
        doc.build(template)
    except:
        failed.append(uri)

In [17]:
# another variety

def do_thing_without_exception(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    split_strings = [i.replace('</p>', '') for i in split_strings]
    for num, i in enumerate(split_strings):
        for tag in ('b', 'i', 'a'):  # in case of extra single-letter </tag> in string
            extra_tag = i.count('<{}>'.format(tag)) - i.count('</{}>'.format(tag))
            if extra_tag < 0:
                for _ in range(extra_tag):
                    print(split_strings)
                    last_pos = i.rfind('</{}>'.format(tag))
                    split_strings[num] = i[:last_pos] + i[last_pos+3:]
                    extra_closed_tag.append((uri, i))
            if extra_tag > 0:
                for _ in range(extra_tag):
                    split_strings[num] = i + '</{}>'.format(tag)
    if len(split_strings) == 1:
        p = Paragraph(split_strings[0], styles["Justify"])
        template.append(p)
    else:
        for i in split_strings:
            if len(i):
                p = Paragraph(i, styles["Justify"])
                template.append(p)
                template.append(Spacer(1,0.2*inch))
            else:
                pass
                print('Empty string: {}'.format(i))
    doc.build(template)


In [12]:
# This one prints to screen each step, to nail down when the data error is
# Only run this one as 'do_thing_print_fails('25322')
#   (E.g., for one item, else it will use all your RAM and freeze your computer)

def do_thing_print_fails(uri):
    issue = cwbr_dict[uri]
    print('a')
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    print('b')
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    print('c')
    try:
        if len(split_strings) == 1:
            print('d')
            p = Paragraph(split_strings[0], styles["Justify"])
            print('e')
            template.append(p)
            print('f')
        else:
            print('g')
            for i in split_strings:
                print('h')
                if len(i):
                    print('i')
                    print(i)
                    p = Paragraph(i, styles["Justify"])
                    print('j')
                    template.append(p)
                    print('k')
                    template.append(Spacer(1,0.2*inch))
                    print('l')
                else:
                    print('m')
                    pass
        print('n')
        doc.build(template)
        print('o')
    except:
        print('p')

In [15]:
# this runs all the items in the loaded csv, with logging fails into a list called "failed".

failed = []

for uri in cwbr_dict:
    do_thing_log_fails(uri)

In [16]:
# prints how many failed, how many are in the loaded csv, and what percent failed.

print(len(failed),
      len(cwbr_dict),
      '{}%'.format(len(failed)/len(cwbr_dict)//.01))


29 29 99.0%


In [18]:
import sys

extra_closed_tag = []
parse_error = []

for uri in failed:
    try:
        do_thing_without_exception(uri)
    except Exception:
        parse_error.append((uri, sys.exc_info()))


print(len(failed))
if parse_error:
    print(parse_error[0])

# with open('exception.txt', 'w') as f:
#     output_text = ''
#     for item in parse_error:
#         output_text += '{}\n'.format(item)
#     f.write(output_text)



29
('5737', (<class 'ValueError'>, ValueError('invalid attribute name target attrMap=[\'backColor\', \'backcolor\', \'bgcolor\', \'color\', \'fg\', \'fontName\', \'fontSize\', \'fontname\', \'fontsize\', \'href\', \'name\', \'textColor\', \'textcolor\']\nparagraph text \'<para><h3><a href="audio/MeierInterview.mp3" target="_blank">Click here for the audio version of the interview</a></h3> <h3><a href="http://www.cwbr.com/index.php?q=5729&field=ID&browse=yes&record=full&searching=yes&Submit=Search">Click here for the review</a></h3> <i><b></i></para>\' caused exception',), <traceback object at 0x7f1d82e87b88>))


In [19]:
# this identifies 3 causes of errors in parsing the data -- and a Leftovers group for items with unknown error

Review_starts_with_url = []
Review_starts_with_image = []
Review_starts_with_header_url = []
Leftovers = []
Other_breaking_types = ['5738', '6050', '3326', '6176', '5804', '3453', ]


for item in parse_error:
#     if item[0] in Other_breaking_types:
#         continue
    if "<para><a href=" in item[1][1].args[0]:
        Review_starts_with_url.append(item)
    elif "<para><img " in item[1][1].args[0]:
        Review_starts_with_image.append(item)
    elif "<para><h3><a href=" in item[1][1].args[0]:
        Review_starts_with_header_url.append(item)
    else:
        Leftovers.append(item)

In [20]:
# prints the number of items in each category.  You want the numbers to end up "0" eventually.

print("failed:", len(failed), '\n',
      "parse_error", len(parse_error), '\n',
      "Review_starts_with_url", len(Review_starts_with_url), '\n',
      "Review_starts_with_header_url", len(Review_starts_with_header_url), '\n',
      "Review_starts_with_image:", len(Review_starts_with_image), "\n",
      "Remainder", len(parse_error)-len(Review_starts_with_url)-len(Review_starts_with_header_url), '\n',
      "Leftovers", len(Leftovers), '\n',) 

print(sorted(failed))

failed: 29 
 parse_error 29 
 Review_starts_with_url 0 
 Review_starts_with_header_url 29 
 Review_starts_with_image: 0 
 Remainder 0 
 Leftovers 0 

['4053', '4245', '4327', '4395', '4481', '4566', '4637', '4755', '4827', '4900', '4970', '5041', '5122', '5207', '5284', '5362', '5437', '5512', '5668', '5737', '5801', '5864', '5928', '5986', '6048', '6113', '6173', '6240', '6306']


In [21]:
# this command is useful now for finding the error in each 'nnnn' listed above.  The error is in the last printed line.

# do_thing_print_fails('4244')

In [23]:
# to see if all of an error type occur in one Record_type.
# replace Review_starts_with_image with whatever error you wish to examine.

url_starts = {"Annotation": 0, "Classics": 0, "Review": 0, "": 0, "Interview": 0}
for i in Review_starts_with_image:
    url_starts[cwbr_dict[i[0]].Record_type] += 1

if Review_starts_with_image:
    print(Review_starts_with_image[0])
    
print(url_starts)
    

{'': 0, 'Classics': 0, 'Annotation': 0, 'Interview': 0, 'Review': 0}
